In [1]:
import pandas as pd

In [2]:
claim = pd.read_excel('claim_summary1.xlsx')

In [3]:
claim_1 = claim.copy()

In [4]:
claim_1.columns

Index(['Video ID', 'Content Type', 'Policy', 'Video Title',
       'Video Duration (sec)', 'Username', 'Uploader', 'Channel Display Name',
       'Channel ID', 'Claim Type', 'Claim Origin', 'Multiple Claims?',
       'Category', 'Asset ID', 'Asset Labels', 'Asset Channel ID', 'Custom ID',
       'Asset Title', 'Season', 'Episode Title', 'Episode Number', 'TMS',
       'Owned Views', 'Owned Views : Watch Page',
       'Owned Views : Embedded Player',
       'YouTube Revenue Split : DoubleClick Served Partner Sold',
       'YouTube Revenue Split : Partner Served Partner Sold',
       'YouTube Revenue Split', 'Partner Revenue'],
      dtype='object')

In [5]:
claim_1 = claim_1[['Content Type','Video Title','Channel Display Name','Asset Labels','Owned Views'
                   ,'YouTube Revenue Split','Partner Revenue']]

In [6]:
claim_1 = claim_1.rename(columns={'Content Type':'content_type','Video Title':'video_title','Channel Display Name':'channel_display_name',
                                  'Asset Labels':'asset_labels','Owned Views':'owned_views',
                                  'YouTube Revenue Split':'youtube_revenue','Partner Revenue':'partner_revenue'})

In [7]:
claim_1 = claim_1.fillna(0)

In [8]:
claim_1

,content_type,video_title,channel_display_name,asset_labels,owned_views,youtube_revenue,partner_revenue
0,UGC,0.0,0.0,AA|BB,44451,53.806053,29.596671
1,Partner-provided,0.0,0.0,AA,11911,132.844946,40.025317
2,UGC,0.0,0.0,AA|CC,591,3.670739,2.018990
3,0,0.0,0.0,0,338,3.551280,0.957647


Separate content types: UGC and Partner-Provided

In [9]:
ugc = claim_1[claim_1['content_type'] == 'UGC']
partner = claim_1[claim_1['content_type'] == 'Partner-provided']

find out all labels of the shows in asset label column

In [10]:
ugc['asset_labels'].unique()

array(['AA|BB', 'AA|CC'], dtype=object)

# separate UGC assets into 2 groups: blank assets and named assets

In [11]:
ugc_blank = ugc[ugc['asset_labels'] == 0]
#ugc_blank = ugc_blank.groupby(['asset_labels','content_type']).sum().reset_index()
ugc_blank

,content_type,video_title,channel_display_name,asset_labels,owned_views,youtube_revenue,partner_revenue


In [12]:
ugc_fill = ugc[ugc['asset_labels'] !=0]
ugc_fill['partner_revenue'].sum()

31.615661

In [13]:
ugc_blank['partner_revenue'].sum() + ugc_fill['partner_revenue'].sum()

31.615661

double check asset labels column again for possible uncorrect/replace asset label

# start to break the assets with "|" for named assets and even out their views, youtube rev. and partner rev. 

In [14]:
# Caculate how many asset ID are in each cell
df_process= ugc_fill["asset_labels"].str.split("|", n = 40, expand = True)  #split by |
df_process.fillna(value=pd.np.nan, inplace=True)  # fill none with na
df_process = df_process.fillna(1)  # fill na with 1
df_process = df_process.apply(pd.to_numeric,errors='coerce')  # change string to numrical, in this case, string will be na
df_process['total'] = df_process.sum(axis=1) #sum how many "1"
df_process['denominator'] =  len(df_process.columns) - df_process['total']-1  # how many assets IDs
df_process.head()

,0,1,total,denominator
0,NaN,NaN,0.0,2.0
2,NaN,NaN,0.0,2.0


In [16]:
colum_name = list(ugc_fill.columns)[4:7]
colum_name

['owned_views', 'youtube_revenue', 'partner_revenue']

In [17]:
## Avg the columns have to be averaged
colum_name = list(ugc_fill.columns)[4:7]
for n in colum_name:
    ugc_fill[n + ' avg']  = ugc_fill[n]/df_process['denominator']


ugc_fill.drop(ugc.columns[4:7], axis = 1, inplace = True) ## drop off the un-avg column, we don't need them
ugc_fill['Number_AssetID'] = df_process['denominator']  ## add the number of asset IDs in one cell in this avg df
ugc_fill.head()

,content_type,video_title,channel_display_name,asset_labels,owned_views avg,youtube_revenue avg,partner_revenue avg,Number_AssetID
0,UGC,0.0,0.0,AA|BB,22225.5,26.903026,14.798336,2.0
2,UGC,0.0,0.0,AA|CC,295.5,1.835370,1.009495,2.0


Breaking down multiple Asset IDs

In [18]:
#select rows need to be broke down
array = ugc_fill[ugc_fill['Number_AssetID']>1]   ## this are df need to be broke down
df_stay = ugc_fill[ugc_fill['Number_AssetID']<=1] 
array.head()

,content_type,video_title,channel_display_name,asset_labels,owned_views avg,youtube_revenue avg,partner_revenue avg,Number_AssetID
0,UGC,0.0,0.0,AA|BB,22225.5,26.903026,14.798336,2.0
2,UGC,0.0,0.0,AA|CC,295.5,1.835370,1.009495,2.0


In [19]:
# define the function the how to break the row 
def fixRow (row):
    rows = pd.DataFrame()
    for item in row['asset_labels'].split('|'):
        df_newrow = row
        df_newrow['asset_labels'] = item
        rows = rows.append(df_newrow)
    return rows

In [20]:
# put the function back into the rows
blank = pd.DataFrame()

for i in range(len(array)):
    if i % 1000 == 0:
        print(i)
    blank = blank.append(fixRow(array.iloc[i]))
    
blank.head()

#fixRow(array.iloc[1]).append(fixRow(array.iloc[0])).head()
    #blank = blank.append()

0


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Number_AssetID,asset_labels,channel_display_name,content_type,owned_views avg,partner_revenue avg,video_title,youtube_revenue avg
0,2.0,AA,0.0,UGC,22225.5,14.798336,0.0,26.903026
0,2.0,BB,0.0,UGC,22225.5,14.798336,0.0,26.903026
2,2.0,AA,0.0,UGC,295.5,1.009495,0.0,1.835370
2,2.0,CC,0.0,UGC,295.5,1.009495,0.0,1.835370


In [21]:
blank['partner_revenue avg'].sum()

31.615661000000003

In [22]:
df_stay['partner_revenue avg'].sum()

0.0

In [23]:
#Combine the "stay" file and the new broke down file. Generate this new mastersheet
new_ugc = df_stay.append(blank)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [24]:
new_ugc['partner_revenue avg'].sum()

31.615661000000003

In [ ]:
new_ugc = new_ugc.rename(columns={'owned_views avg':'owned_views',
                                  'partner_revenue avg':'partner_revenue',
                                  'youtube_revenue avg':'youtube_revenue'})

In [ ]:
new_ugc = new_ugc[['content_type', 'channel_display_name', 'asset_labels',
       'owned_views', 'youtube_revenue', 'partner_revenue']]

In [ ]:
new_ugc = new_ugc.groupby(['asset_labels','content_type']).sum().reset_index()

In [ ]:
ugc_group = new_ugc.append(ugc_blank)
ugc_group

In [ ]:
ugc_group['partner_revenue'].sum()

In [ ]:
partner_group = partner.groupby(['channel_display_name','content_type']).sum().reset_index()

In [ ]:
partner_group = partner_group.rename(columns={'channel_display_name':'asset_labels'})
partner_group

In [ ]:
final_combined = pd.DataFrame()

final_combined = final_combined.append(ugc_group,ignore_index=True)
final_combined = final_combined.append(partner_group,ignore_index=True)

In [ ]:
final_combined.reset_index()

In [ ]:
final_combined['partner_revenue'].sum()

In [ ]:
final_combined.to_csv('ad_partner_rev.csv')